# import

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from collections import OrderedDict
from scipy.spatial import distance as dist
import cv2


ModuleNotFoundError: No module named 'matplotlib'

In [2]:
def binary(x):
    # transform an image in black and white
    if x == 0:
        return 0
    else:
        return 255

In [3]:
############## 해석 필요 ####################
def update_boxes(box):
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    return x, y, x + w, y + h

In [4]:
############## 해석 필요 ####################
def remove_ball_boy(detected_person_img, lower_col, upper_col):
    # compute pixel percentage of a range of color (lower_col, upper_col)
    # in each box to detect ball boys/girls
    mask = cv2.inRange(detected_person_img, lower_col, upper_col)
    img = cv2.bitwise_and(detected_person_img, detected_person_img, \
                          mask=mask)
    func = np.vectorize(binary)
    img = func(img).astype(np.uint8)

    n_pix = img.shape[0] * img.shape[1]
    n_pix_bb = sum(img.flatten()) / img.shape[2] / 255

    return n_pix_bb / n_pix

In [5]:
# YOLO를 포함한 CNN 모델은 여러 개의 layer를 가지고 있다.
# 그중 결과 값이 들어있는 layers만 선별해서 사용하기 위한 코드
def get_output_layer(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

In [6]:
############## 해석 필요 #################### -> 여기서 문제가 생김
def predict_players(outs, LABELS, img, confidence_threshold=0.8):
    class_ids = []
    confidences = []
    boxes = []
    confi_threshold = 0.8
    nms_threshold = 0.00000000001
    Width = img.shape[1]
    Height = img.shape[0]
    predicted_players = []

    print("outs: ",outs)

    # dataFrame = pd.DataFrame(outs)
    # print(dataFrame)
    for out in outs:
        print("out: ",out)
        for detection in out:
            # print("2",detection)
            scores = detection[5:]
            # print("3) scores", scores)
            # np.argmax: 최대값의 인덱스 번호(색인 위치)
            class_id = np.argmax(scores)
            # print("class_id",class_id)
            confidence = scores[class_id]
            # print("confidence",confidence)
            if confidence > confi_threshold and LABELS[class_id] == 'person':
                print("**************")
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                taux = remove_ball_boy(img[y:y + h, x:x + w], \
                                                    (27, 5, 40), (47, 40, 168))

                if taux < 0.01:
                    class_ids.append(class_id)
                    confidences.append(float(confidence))
                    boxes.append([x, y, w, h])

        # apply non-maxima suppression to suppress weak, overlapping
        # bounding boxes
        idxs = cv2.dnn.NMSBoxes(boxes, confidences, confi_threshold,
                                nms_threshold)

        # ensure at least one detection exists
        if len(idxs) > 0:
            # loop over the indexes we are keeping
            for i in idxs.flatten():
                # extract the bounding box coordinates
                (x, y) = (boxes[i][0], boxes[i][1])
                (w, h) = (boxes[i][2], boxes[i][3])

                predicted_players.append((x, y, w, h))

        # sometimes, two identical frames remain
        predicted_players = list(set(predicted_players))

        return predicted_players

In [7]:
# YOLO를 포함한 CNN 모델은 여러 개의 layer를 가지고 있다.
# 그중 결과 값이 들어있는 layers만 선별해서 사용하기 위한 코드
def get_output_layer(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers


In [8]:
class CentroidTracker:
    def __init__(self, maxDisappeared=300):
        self.nextObjectID = 0
        self.objects = OrderedDict()  # 추적했을 때 선수 object
        self.disappeared = OrderedDict()  # 추적하지 못한 프레임 표시?
        self.maxDisappeared = maxDisappeared  # 프레임 밖으로 나가도 인정해주는 시간

    def register(self, centroid):
        # when registering an object we use the next available object
        # ID to store the centroid
        print("register")
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1

    def deregister(self, objectID):
        # to deregister an object ID we delete the object ID from
        # both of our respective dictionaries
        print("deregister")
        del self.objects[objectID]
        del self.disappeared[objectID]

    ############## 해석 필요 ####################
    def update(self, rects):
        # rects 가 비어있는지 확인
        if len(rects) == 0:
            # loop over any existing tracked objects and mark them
            # as disappeared
            # 기존 트랙킹된 object를 반복하여 사라진 것으로 표시
            for objectID in list(self.disappeared.keys()):
                print("objectID")
                self.disappeared[objectID] += 1
                # if we have reached a maximum number of consecutive
                # frames where a given object has been marked as
                # missing, deregister it
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)
                # return early as there are no centroids or tracking info
                # to update
            print("len(rects)==0", self.objects)
            return self.objects

        # initialize an array of input centroids for the current frame
        inputCentroids = np.zeros((len(rects), 2), dtype="int")
        # loop over the bounding box rectangles
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            # use the bounding box coordinates to derive the centroid
            cX = int((startX + endX) / 2.0)
            cY = int(endY)
            inputCentroids[i] = (cX, cY)

        # if we are currently not tracking any objects take the input
        # centroids and register each of them
        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])

        # otherwise, are are currently tracking objects so we need to
        # try to match the input centroids to existing object
        # centroids
        else:
            # grab the set of object IDs and corresponding centroids
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())
            # compute the distance between each pair of object
            # centroids and input centroids, respectively -- our
            # goal will be to match an input centroid to an existing
            # object centroid
            D = dist.cdist(np.array(objectCentroids), inputCentroids)
            # in order to perform this matching we must (1) find the
            # smallest value in each row and then (2) sort the row
            # indexes based on their minimum values so that the row
            # with the smallest value is at the *front* of the index
            # list
            rows = D.min(axis=1).argsort()
            # next, we perform a similar process on the columns by
            # finding the smallest value in each column and then
            # sorting using the previously computed row index list
            cols = D.argmin(axis=1)[rows]

            # in order to determine if we need to update, register,
            # or deregister an object we need to keep track of which
            # of the rows and column indexes we have already examined
            usedRows = set()
            usedCols = set()
            # loop over the combination of the (row, column) index
            # tuples
            for (row, col) in zip(rows, cols):
                # if we have already examined either the row or
                # column value before, ignore it
                # val
                if row in usedRows or col in usedCols:
                    continue
                # otherwise, grab the object ID for the current row,
                # set its new centroid, and reset the disappeared
                # counter
                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0
                # indicate that we have examined each of the row and
                # column indexes, respectively
                usedRows.add(row)
                usedCols.add(col)

            # compute both the row and column index we have NOT yet
            # examined
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            # in the event that the number of object centroids is
            # equal or greater than the number of input centroids
            # we need to check and see if some of these objects have
            # potentially disappeared
            if D.shape[0] >= D.shape[1]:
                # loop over the unused row indexes
                for row in unusedRows:
                    # grab the object ID for the corresponding row
                    # index and increment the disappeared counter
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1
                    # check to see if the number of consecutive
                    # frames the object has been marked "disappeared"
                    # for warrants deregistering the object
                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)

            # otherwise, if the number of input centroids is greater
            # than the number of existing object centroids we need to
            # register each new input centroid as a trackable object
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col])
        # return the set of trackable objects
        print("len(rects)!=0", self.objects)
        return self.objects

In [9]:
current_frame = 0
width, height = 640, 360
img, img1, img2 = None, None, None
tracknet_width, tracknet_height = 640, 360

# 영상불러오기 및 영상정보 추출
video = cv2.VideoCapture('video/input_video.mp4')
fps = int(video.get(cv2.CAP_PROP_FPS))
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# 영상정보 출력
print('fps : {}'.format(fps))
print('frame sizee : {}x{}'.format(frame_width, frame_height))
print('num_frames :{}'.format(num_frames))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter('video/video_output.mp4', fourcc, fps, (frame_width, frame_height))

fps : 23
frame sizee : 854x480
num_frames :283


In [11]:
# yolov3
# 라벨링한다 -> 변수정의라 생각함
LABELS = open("Yolov3/yolov3.txt").read().strip().split("\n")
# 네트워크 불러오기 -> opencv로 딥러닝을 실행하기 위해 생성
net = cv2.dnn.readNet("Yolov3/yolov3.weights", "Yolov3/yolov3.cfg")

ct_players = CentroidTracker()

# append players positions at each frame
players_positions = {'x_0': [], 'y_0': [], 'x_1': [], 'y_1': []}


In [14]:

while (True):
    print("current frame", current_frame)
    video.set(1, current_frame)
    ret, img = video.read()
    if not ret:
        break

    scale = 0.00392
    # 입력 영상을 blob 객체로 만들기 -> 해당 인자들은 모델 파일의 학습에 맞게 입력되어있음
    # blob: Binary Large Object 의 약자. 즉, 바이너리 형태로 큰 객체(이미지, 사운드 등)를 저장
    blob = cv2.dnn.blobFromImage(img, scale, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)  # 네트워크 입력 설정
    # 네트워크 순방향 실행을 위한 코드
    # input: 출력 레이어 이름 리스트
    # output: 지정한 레이어의 출력 블롭 리스트
    outs = net.forward(get_output_layer(net))
    print("outs",outs)
    detected_players = predict_players(outs, LABELS, img, 0.8)
    print(detected_players)

    ############## 해석 필요 ####################

    # map 함수는 첫번째 매개변수에 함수, 두번째 매개변수에 반복 가능한 자료형(리스트, 튜플 등)
    # map 함수의 반환 값은 map 객체-> 해당 자료형을 list 혹은 tuple 로 형변환 필요
    # map(적용시킬 함수, 적용할 값들)
    # track players with a unique ID
    formate_detected_players = list(map(update_boxes, list(detected_players)))
    # print("formate_detected_players: ", formate_detected_players)
    players_objects = ct_players.update(formate_detected_players)
    print(players_objects.items())

    # players positions frame by frame

    # players_positions['x_0'].append(tuple(players_objects[0])[0])
    # players_positions['y_0'].append(tuple(players_objects[0])[1])
    # players_positions['x_1'].append(tuple(players_objects[1])[0])
    # players_positions['y_1'].append(tuple(players_objects[1])[1])

    output_img = img
    PIL_image = cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB)
    PIL_image = Image.fromarray(PIL_image)

    # draw players boxes
    color_box = (0, 0, 255)
    if len(detected_players) > 0:
        for box in detected_players:
            x, y, w, h = box
            cv2.rectangle(output_img, (x, y), (x + w, y + h), color_box, 2)

    # draw tracking id of each player
    for (objectID, centroid_player)in players_objects.items():
        # draw both the ID of the object and the centroid of the
        # object on the output frame
        text = "ID {}".format(objectID)
        cv2.putText(output_img, text, (centroid_player[0] - 50, centroid_player[1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
        cv2.circle(output_img, (centroid_player[0], centroid_player[1]), 1, (0, 255, 0), 2)

    # Convert PIL image format back to opencv image format
    opencvImage = cv2.cvtColor(np.array(PIL_image), cv2.COLOR_RGB2BGR)
    # write image to output_video
    output_video.write(opencvImage)

    # next frame
    current_frame += 1


current frame 0
outs [array([[0.04700029, 0.04409599, 0.45061398, ..., 0.        , 0.        ,
        0.        ],
       [0.0518713 , 0.0334424 , 0.27834228, ..., 0.        , 0.        ,
        0.        ],
       [0.0465703 , 0.03490556, 0.7586337 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.954375  , 0.95579016, 0.46097502, ..., 0.        , 0.        ,
        0.        ],
       [0.96004426, 0.96217036, 0.301554  , ..., 0.        , 0.        ,
        0.        ],
       [0.9711673 , 0.9608296 , 0.8934141 , ..., 0.        , 0.        ,
        0.        ]], dtype=float32), array([[0.02459472, 0.02521133, 0.05260286, ..., 0.        , 0.        ,
        0.        ],
       [0.01955835, 0.02313578, 0.31064403, ..., 0.        , 0.        ,
        0.        ],
       [0.02316862, 0.01852349, 0.07603523, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.9731328 , 0.9818334 , 0.04197319, ..., 0.        , 0.        ,
        0.        

NameError: name 'Image' is not defined